# Interactive plot of simple vector features (points) with Bokeh and GeoPandas

Data used: Properties in Care in Scotland (<https://portal.historicenvironment.scot/downloads/propertiesincare>)

In [2]:
# import libraries
import geopandas as gpd
from bokeh.io import output_notebook
from bokeh.models import CategoricalColorMapper, GeoJSONDataSource
from bokeh.palettes import viridis
from bokeh.plotting import figure, show
from bokeh.tile_providers import CARTODBPOSITRON_RETINA, get_provider

In [3]:
# set inline plots
output_notebook()

Loading BokehJS ...

In [4]:
# specify map tile provider
tile_provider = get_provider(CARTODBPOSITRON_RETINA)

In [2]:
# import data
data = gpd.read_file("data/pic/properties_in_care.shp")

In [3]:
# view data
data.head(5)

,PIC_ID,PIC_NAME,X,Y,LINK,LOCAL_AUTH,geometry
0,PIC001,Aberdour Castle,319282.984834,685434.235721,https://www.historicenvironment.scot/visit-a-p...,Fife,POINT (319282.979 685434.233)
1,PIC002,Aberlemno Churchyard Cross Slab,352238.075325,755559.199414,https://www.historicenvironment.scot/visit-a-p...,Angus,POINT (352238.075 755559.199)
2,PIC003,Aberlemno Sculptured Stones,352260.213801,755900.088942,https://www.historicenvironment.scot/visit-a-p...,Angus,POINT (352260.214 755900.089)
3,PIC004,Abernethy Round Tower,318992.456831,716391.678420,https://www.historicenvironment.scot/visit-a-p...,Perth and Kinross,POINT (318992.457 716391.678)
4,PIC005,Affleck Castle,349401.057104,738809.990123,None,Angus,POINT (349401.057 738809.990)


In [6]:
# reproject to web mercator
data = data.to_crs(3857)

In [7]:
# convert data source to GeoJSON
geo_source = GeoJSONDataSource(geojson=data.to_json())

In [8]:
# generate unique colours for each local authority
const = list(set(data["LOCAL_AUTH"]))
palette = viridis(len(const))
color_map = CategoricalColorMapper(factors=const, palette=palette)

In [9]:
# define title and tooltips
TITLE = (
    "Properties in Care in Scotland. © Historic Environment Scotland 2021."
)

TOOLTIPS = [
    ("NAME", "@PIC_NAME"),
    ("LOCAL_AUTH", "@LOCAL_AUTH"),
    ("COORDINATES", "(@X, @Y)"),
    ("ID", "@PIC_ID")
]

In [10]:
# configure plot
p = figure(
    title=TITLE,
    tools="wheel_zoom, pan, reset, hover, save",
    x_axis_location=None,
    y_axis_location=None,
    tooltips=TOOLTIPS,
    x_axis_type="mercator",
    y_axis_type="mercator"
)

p.grid.grid_line_color = None

p.hover.point_policy = "follow_mouse"

p.circle(
    "x",
    "y",
    source=geo_source,
    size=5,
    line_width=0,
    fill_color={"field": "LOCAL_AUTH", "transform": color_map}
)

p.add_tile(tile_provider)

# display plot
show(p)